In [33]:
# !pip install tensorflow
# !pip install --upgrade torch
!pip install onnx
# !pip install onnxruntime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 13.5 MB 7.3 MB/s            
     |████████████████████████████████| 1.0 MB 52.5 MB/s            
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.6.0 requires numpy~=1.19.2, but you have numpy 1.24.2 which is incompatible.
tensorflow-gpu 2.6.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow-gpu 2

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from time import perf_counter
import tqdm 
from transformers import pipeline

In [10]:
from transformers.convert_graph_to_onnx import convert
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModel,BartForConditionalGeneration

In [3]:
model_ckpt= 'circulus/kobart-trans-ko-en-v2' #bart`

In [4]:
directory = os.getenv('HOME') + '/aiffel/aiffelthon/meta'

In [5]:
os.listdir(directory)

['reg_1.json',
 '.ipynb_checkpoints',
 'kor_text.xlsx',
 'reg_3.json',
 'meta_data_included_raw.csv',
 'meta_data_raw_eng_match_6m.csv',
 'kor_text.txt',
 'reg_4.json',
 'reg_5.json',
 'reg_2.json']

In [ ]:
# df = pd.read_csv(directory + '/meta_data_included_raw.csv')

In [7]:
os.listdir(os.getenv('HOME')+'/aiffel/aiffelthon/final')

['Train_set_long_sequence_finalv1.csv',
 'raw_data_sampling(0223).csv',
 'Train_set_data.csv',
 'Test_set_data_sampling(0223).csv',
 'Train_set_long_sequence.csv']

In [9]:
dt = pd.read_csv(os.getenv('HOME') + '/aiffel/aiffelthon/final/Test_set_data_sampling(0223).csv')

In [12]:
# dm = df.merge(dt, left_on='dial',right_on='dial')

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, max_len=256, return_token_type_ids=False, model_input_names='input_ids', return_tensors='pt')
model = BartForConditionalGeneration.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/304 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [114]:
import torch
emb = tokenizer('안녕  ', padding='max_length')['input_ids']

In [115]:
tokenizer

PreTrainedTokenizerFast(name_or_path='circulus/kobart-trans-ko-en-v2', vocab_size=30000, model_max_len=256, is_fast=True, padding_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'})

In [16]:
model(torch.tensor([[22465]]), decoder_input_ids=torch.tensor([[1,16391,304]])).logits

tensor([[[ -6.3160,   6.0853,  -7.6853,  ...,  -7.5236,  -7.0923,  -8.6508],
         [ -7.7646,   6.4150,  -8.8871,  ...,  -8.5864, -10.1093,  -7.6269],
         [ -9.9529,  12.4529, -10.5425,  ...,  -9.9482, -10.0135, -12.1245]]],
       grad_fn=<AddBackward0>)

In [20]:
os.makedirs(os.getenv('HOME') + '/aiffel/aiffelthon/onnx/', exist_ok=True)

In [21]:
from pathlib import Path
onnx_model_path = os.getenv('HOME') + '/aiffel/aiffelthon/onnx/model_trans.onnx'

In [22]:
dummy_input_1 = torch.randint(1,100,(1,256))
dummy_input_2 = torch.randint(1,100,(1,256))

# This is how we would call the PyTorch model
example_output = model(dummy_input_1, dummy_input_2)

# This is how to export it with multiple inputs
torch.onnx.export(model,
        args=(dummy_input_1, dummy_input_2),
        f=onnx_model_path,
        opset_version = 12,
        input_names=["encoder_input", "decoder_input"],
        output_names=["output1"])

In [48]:
from onnxruntime import (GraphOptimizationLevel, InferenceSession, 
                         SessionOptions)

def create_model_for_provider(model_path, provider="CPUExecutionProvider"): 
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    session = InferenceSession(str(model_path), options, providers=[provider])
    session.disable_fallback()
    return session

In [49]:
onnx_model = create_model_for_provider(onnx_model_path)

In [94]:
from scipy.special import softmax

class OnnxPipeline:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __call__(self, query):
        model_inputs = self.tokenizer(query, return_tensors="pt", padding='max_length')
        model_inputs2 = self.tokenizer('</s>', return_tensors="pt", padding='max_length')
        inputs_onnx = {'encoder_input': v.cpu().detach().numpy() 
                       for k, v in model_inputs.items()}
        inputs_onnx.update({'decoder_input':model_inputs2['input_ids'].cpu().detach().numpy()})
        logits = self.model.run(None, inputs_onnx)[0]
        probs = softmax(logits)
        pred_idx = np.argmax(probs).item()
        pred_txt = tokenizer.decode(pred_idx)
        
#         for _ in range(10) :
#             inputs_onnx['decoder_input'] = self.tokenizer(decoder_txt, return_tensors="pt", padding='max_length')['input_ids'].cpu().detach().numpy()
#             logits = self.model.run(None, inputs_onnx)[0]
#             probs = softmax(logits)
#             pred_idx = np.argmax(probs).item()
#             pred_txt = tokenizer.decode(pred_idx)
            
#             decoder_txt += pred_txt
            
        return pred_idx

In [110]:
query = '안녕하세요 제 이름은'
pipe = OnnxPipeline(onnx_model, tokenizer)
pipe(query)

16391

In [111]:
tokenizer.batch_decode([16391])

['H']

In [116]:
emb

[22465,
 1700,
 1700,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3

In [117]:
model.generate(emb)

AttributeError: 'list' object has no attribute 'ne'

In [35]:
from onnxruntime.quantization import quantize_dynamic, QuantType

model_input = os.getenv("HOME")+'/aiffel/aiffelthon/onnx/model_trans.onnx'
model_output = os.getenv("HOME")+'/aiffel/aiffelthon/onnx/model_trans.quant.onnx'
quantize_dynamic(model_input, model_output, weight_type=QuantType.QInt8)

Ignore MatMul due to non constant B: /[MatMul_217]
Ignore MatMul due to non constant B: /[MatMul_232]
Ignore MatMul due to non constant B: /[MatMul_326]
Ignore MatMul due to non constant B: /[MatMul_341]
Ignore MatMul due to non constant B: /[MatMul_435]
Ignore MatMul due to non constant B: /[MatMul_450]
Ignore MatMul due to non constant B: /[MatMul_544]
Ignore MatMul due to non constant B: /[MatMul_559]
Ignore MatMul due to non constant B: /[MatMul_653]
Ignore MatMul due to non constant B: /[MatMul_668]
Ignore MatMul due to non constant B: /[MatMul_762]
Ignore MatMul due to non constant B: /[MatMul_777]
Ignore MatMul due to non constant B: /[MatMul_978]
Ignore MatMul due to non constant B: /[MatMul_993]
Ignore MatMul due to non constant B: /[MatMul_1063]
Ignore MatMul due to non constant B: /[MatMul_1078]
Ignore MatMul due to non constant B: /[MatMul_1172]
Ignore MatMul due to non constant B: /[MatMul_1187]
Ignore MatMul due to non constant B: /[MatMul_1257]
Ignore MatMul due to non c

In [36]:
onnx_model_q = create_model_for_provider(model_output)

In [70]:
query = '안녕하세요'
pipe = OnnxPipeline(onnx_model_q, tokenizer)
pipe(query)

'<s> '